In [3]:
import os
import sys

def get_current_path():
    """
    Trả về đường dẫn đầy đủ tới file .py hoặc .ipynb hiện tại.
    """
    try:
        # Trường hợp chạy từ file .py
        current_path = os.path.abspath(__file__)
    except NameError:
        # Trường hợp __file__ không tồn tại (ví dụ Jupyter Notebook)
        try:
            import ipykernel
            from notebook import notebookapp
            import urllib.request
            import json

            # Lấy kernel id
            connection_file = os.path.basename(ipykernel.get_connection_file())
            kernel_id = connection_file.split('-', 1)[1].split('.')[0]

            # Duyệt server Jupyter để tìm notebook
            for srv in notebookapp.list_running_servers():
                try:
                    response = urllib.request.urlopen(srv['url'] + 'api/sessions')
                    sessions = json.load(response)
                    for sess in sessions:
                        if sess['kernel']['id'] == kernel_id:
                            return os.path.join(srv['notebook_dir'], sess['notebook']['path'])
                except:
                    continue
            # Nếu không tìm thấy notebook
            return os.getcwd()
        except:
            # Nếu không phải Jupyter, trả về thư mục làm việc
            return os.getcwd()
    return current_path


In [4]:
get_current_path()

'd:\\VPBanks\\VPBanks\\project'

In [19]:
import streamlit as st
import pandas as pd
# from OOP_redshit import *
import plotly.express as px
import hashlib

import os

# Bật chế độ wide mode
# st.set_page_config(layout="wide")

# Lấy dữ liệu 
DIR = get_current_path()
csv_path = os.path.join(DIR, "job_pt.csv")

hd = pd.read_csv(csv_path)
hd['opndate'] = pd.to_datetime(hd['opndate'],format='%Y-%m-%d')
hd['cash_in_date'] = pd.to_datetime(hd['cash_in_date'],format='%Y-%m-%d')
del hd['Unnamed: 0']



In [20]:
hd

,opndate,cash_in_date,custodycd,acctno,cashin,daterank,cashin_datediff
0,2023-11-23,2023-11-23,116C489192,4014290,100000000.0,1,0
1,2024-06-11,2024-06-12,116C149589,1586419,1000000.0,1,1
2,2023-09-26,2024-07-10,116C782567,1279835,1300000.0,1,288
3,2024-12-06,2024-12-06,116C315667,1804719,3000000.0,1,0
4,2023-03-28,2023-03-29,116C701657,2026254,2170000.0,1,1
...,...,...,...,...,...,...,...
25469,2022-12-01,2022-12-05,116C258306,2009210,200000000.0,1,4
25470,2022-11-23,2022-12-09,116C555616,2008191,20000000.0,1,16
25471,2023-11-29,2024-06-06,116C948532,2038943,5000000.0,1,190
25472,2024-02-27,2024-03-22,116C129298,4018112,50000000.0,1,24


In [21]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25474 entries, 0 to 25473
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   opndate          25474 non-null  datetime64[ns]
 1   cash_in_date     25474 non-null  datetime64[ns]
 2   custodycd        25474 non-null  object        
 3   acctno           25474 non-null  int64         
 4   cashin           25474 non-null  float64       
 5   daterank         25474 non-null  int64         
 6   cashin_datediff  25474 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 1.4+ MB


In [35]:
hd_time_year = hd[['opndate','custodycd','cashin','cashin_datediff']]
hd_time_year['yeardf'] = hd_time_year['opndate'].dt.year.astype('str')
hd_time_year = hd_time_year[hd_time_year['cashin'] <= 10*10**9]
hd_time_year

C:\Users\hieudd\AppData\Local\Temp\ipykernel_34460\3280580640.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,opndate,custodycd,cashin,cashin_datediff,yeardf
0,2023-11-23,116C489192,100000000.0,0,2023
1,2024-06-11,116C149589,1000000.0,1,2024
2,2023-09-26,116C782567,1300000.0,288,2023
3,2024-12-06,116C315667,3000000.0,0,2024
4,2023-03-28,116C701657,2170000.0,1,2023
...,...,...,...,...,...
25469,2022-12-01,116C258306,200000000.0,4,2022
25470,2022-11-23,116C555616,20000000.0,16,2022
25471,2023-11-29,116C948532,5000000.0,190,2023
25472,2024-02-27,116C129298,50000000.0,24,2024


In [25]:
hd_time_year.groupby('yeardf').agg(custodycd = ('custodycd','nunique'),
                                   datediff = ('cashin_datediff','mean'),
                                   cashin = ('cashin','mean')
                                   )

,custodycd,datediff,cashin
yeardf,,,
2022,4525,109.964250,1.268938e+08
2023,8028,48.898059,1.649405e+08
2024,6291,30.505698,9.267255e+07
2025,4659,8.623754,1.293490e+08


In [37]:
fig = px.box(hd_time_year, x="cashin", y="yeardf",color='yeardf')
fig.show()

# st.plotly_chart(fig, use_container_width=True)

In [ ]:
# st.title('Thống kê mô tả cơ bản')
# '''
# Mẫu dữ liệu
# '''
# st.table(hd.head(10))

# '''
# Mô tả thống kê
# '''
# hd_sta = hd.describe()

# st.table(hd_sta)

# hd.describe()
# ###############################
# st.title("Phân bổ thời gian chờ khách hàng nạp tiền lần đầu")

# fig = px.histogram(hd[hd['cashin_datediff']<=hd_sta['cashin_datediff']['75%']], x="cashin_datediff")

# st.plotly_chart(fig, use_container_width=True)

# # ###############################
# st.title("Phân bổ giá trị tiền nạp lần đầu của khách hàng")

# fig = px.histogram(hd[hd['cashin']<=hd_sta['cashin']['75%']], x="cashin")

# st.plotly_chart(fig, use_container_width=True)

# # ###############################
# st.title(" Độ tương quan giữa thời gian chờ khách nạp lần đầu vs giá trị tiền nạp lần đầu")

# hd_cash_date = hd[hd['cashin'] <= 10*10**9][['custodycd','cashin','cashin_datediff']]

# # --- Nút chọn dataset với mặc định ---
# option = st.radio(
#     "Chọn dataset để hiển thị:",
#     ("Full", "Filter (max 10B)"),
#     index=0  # 👈 mặc định chọn "Full"
# )

# # --- Lọc dữ liệu và hiển thị chart ---
# if option == "Full":
#     filtered_df = hd
#     chart_title = "📈 Chart Full Dataset"
# else:
#     filtered_df = hd_cash_date
#     chart_title = "📉 Chart Filter with max value 10x10^9 Dataset"
    
# fig = px.scatter(filtered_df, x="cashin_datediff", y="cashin",title= chart_title)

# st.plotly_chart(fig, use_container_width=True)

# # ##############################
# st.title("📊 Số lượng khách hàng nạp tiền vào tài khoản theo khoảng thời gian")

# hd_cust = hd.groupby('cashin_datediff').agg(custodycd_count = ('custodycd','count'))
# hd_cust.reset_index(inplace=True)
# hd_cust_sta = hd_cust.describe()
# hd_cust_sta.reset_index(inplace=True)

# # --- Nút chọn dataset với mặc định ---
# option = st.radio(
#     "Chọn dataset để hiển thị:",
#     ("Full", "10 ngày đầu tiên "),
#     index=0  # 👈 mặc định chọn "Full"
# )

# # --- Lọc dữ liệu và hiển thị chart ---
# if option == "Full":
#     filtered_df = hd_cust
#     chart_title = "📈 Chart Full Dataset"
# else:
#     filtered_df = hd_cust[hd_cust["cashin_datediff"] <= 10]
#     chart_title = "📉 Chart 75% Dataset"

# fig = px.bar(filtered_df, x="cashin_datediff", y="custodycd_count", title= chart_title)

# st.plotly_chart(fig, use_container_width=True)